In [2]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" pip install llama-cpp-python
!pip install sentence-transformers
!pip install llama-index

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 15.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.11-cp310-cp310-manylinux_2_35_x86_64.whl size=6423457 sha256=b05d48dfb9a7e62ce4d97feaf41c65a99ef5d8ab5693b06995d67f86261305c3
  Stored in directory: /root/.cache/pip/wheels/dc/42/77/a3ab0d02700427ea364de5797786c0272779dce795f62c3bc2
Successfully built llama-cpp-python
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 41.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
from langchain.embeddings import HuggingFaceEmbeddings
from llama_index import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    ServiceContext,
)
from llama_index.llms import LlamaCPP
from llama_index.llms.llama_utils import messages_to_prompt, completion_to_prompt

model_url = "https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF/resolve/main/llama-2-13b-chat.Q4_0.gguf"

llm = LlamaCPP(
    # You can pass in the URL to a GGML model to download it automatically
    model_url=model_url,
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    model_path=None,
    temperature=0.1,
    max_new_tokens=1200,
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=3900,
    # kwargs to pass to __call__()
    generate_kwargs={},
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": 43},  #28,29,30 layers works best on my setup.
    # transform inputs into Llama2 format
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True,
)

# response_iter = llm.stream_complete("Can you write me a poem about fast cars?")
# for response in response_iter:
#     print(response.delta, end="", flush=True)

total size (MB): 7365.83


7025it [00:48, 144.10it/s]                          
AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


In [4]:
from llama_index import SimpleDirectoryReader
from llama_index import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    ServiceContext,

)

embed_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2"
)

service_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model=embed_model,
)
documents = SimpleDirectoryReader('./Judgement').load_data()
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

[nltk_data] Downloading package punkt to /tmp/llama_index...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [5]:
query_engine = index.as_query_engine()

In [6]:
#no need to run this cell until you want to add custom files
documents2 = SimpleDirectoryReader('/content/Input').load_data()
index2 = VectorStoreIndex.from_documents(documents2, service_context=service_context)
query_engine2 = index2.as_query_engine()

In [15]:
response = query_engine.query("Give me a summarization of 1953_L_1.txt with facts above 300 words, argument above 250 words, Issue above 150 words and analysis words above 400 in paragraph format use plain text")
print(response)

Llama.generate: prefix-match hit


  Sure, I can help you with that! Here's a summarization of the given context information in paragraph format, with facts, argument, issue, and analysis exceeding the specified word limits:

Facts:
The case concerns an appeal against the judgment of the High Court, which set aside the order of acquittal passed by the Trial Court. The appellant was charged with culpable homicide not amounting to murder under Section 299 of the Indian Penal Code (IPC). The incident in question took place at around 8:45 pm, and the police received information about it at 3:00 am. The appellant submitted that there were shifting times and places of occurrence, and the prosecution's evidence was not reliable. The High Court interfered with the order of acquittal, and the present appeal is against that judgment.

Argument:
Counsel for the appellant submitted that the High Court erred in setting aside the order of acquittal. They argued that there were violations of Section 162 of the Criminal Procedure Code 

In [18]:
import re
from collections import Counter
from bert_score import score



# Define a function to tokenize text into words
def tokenize_text(text):
    # Use regular expressions to split text into words
    words = re.findall(r'\b\w+\b', text.lower())
    return words

# Define functions to compute ROUGE precision, recall, and F1 scores
def rouge_precision(candidate_tokens, reference_tokens):
    candidate_counts = Counter(candidate_tokens)
    reference_counts = Counter(reference_tokens)
    overlap = sum((candidate_counts & reference_counts).values())
    total_candidate_tokens = len(candidate_tokens)
    return overlap / total_candidate_tokens if total_candidate_tokens > 0 else 0

def rouge_recall(candidate_tokens, reference_tokens):
    candidate_counts = Counter(candidate_tokens)
    reference_counts = Counter(reference_tokens)
    overlap = sum((candidate_counts & reference_counts).values())
    total_reference_tokens = len(reference_tokens)
    return overlap / total_reference_tokens if total_reference_tokens > 0 else 0

def rouge_f1(precision, recall):
    return (2.0 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

# Load the content of the file
with open("1953_L_1.txt", "r") as file:
    text = file.read()

# Example candidate and reference texts
candidate = text
response = str(response)

# Tokenize the candidate and reference texts
candidate_tokens = tokenize_text(candidate)
reference_tokens = tokenize_text(response)

# Calculate ROUGE scores
precision_score = rouge_precision(candidate_tokens, reference_tokens)
recall_score = rouge_recall(candidate_tokens, reference_tokens)
f1_score = rouge_f1(precision_score, recall_score)

print(f'ROUGE Precision Score: {precision_score}')
print(f'ROUGE Recall Score: {recall_score}')
print(f'ROUGE F1 Score: {f1_score}')


P, R, F1 = score([candidate], [response], lang="en")  # Specify the language as "en" (English)
print(f'BERTScore Precision: {P.item():.6f}')
print(f'BERTScore Recall: {R.item():.6f}')
print(f'BERTScore F1: {F1.item():.6f}')


ROUGE Precision Score: 0.15058479532163743
ROUGE Recall Score: 0.4963855421686747
ROUGE F1 Score: 0.23107122826696577


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERTScore Precision: 0.770551
BERTScore Recall: 0.797856
BERTScore F1: 0.783966


In [ ]:
!pip install bert-score
!bert-score-install --lang en

